# PL access to PS DRAM 

In this notebook, the PYNQ allocate class will be used to allocate a memory buffer in the DDR memory. The physical address of the memory will be passed to the PL, in this case to an IOP. The IOP has a connection to the PS DRAM. An application will run on the IOP to modify the contents of the memory buffer in the PS DRAM. 

In a similar way, another IP in the PL can use a physical memory pointer to access PS DRAM. 

## Allocate memory buffer
The allocate class creates a contiguous memory buffer. The buffer can be used from Python. Python is running on Linux, and will access the buffer via a virtual memory address. This is transparent to the user at the Python level. The `.physical_address` allocate attribute can be used to return the physical address. This physical address can be used by IP in the PL to access the same memory buffer in PS DRAM. 

In [1]:
from pynq import allocate
import numpy as np 
py_buffer = allocate(shape=(1000,), dtype=np.int32)

### Check the memory buffer addresses

The virtual address can be used by any application running in Linux. This could be a Python application, or a C/C++ or other application running in Linux. The Physical address can be passed to an IP block in an overlay.

In [2]:
print("py_buffer physical address {}".format(hex(py_buffer.physical_address)))

py_buffer physical address 0x16849000


### Download the base overlay

In [3]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

## Create MicroBlaze program

The C code for a new function that will run on a MicroBlaze is provided in the next cell. The C function parameters are a physical address, a length, and data. The function will modify the contents of the memory. It will modify data in the range [*address* : *address*+*length*], by reading the contents of each memory location, and adding an offset value *data* to each location.

In [4]:
%%microblaze base.ARDUINO
void my_function(unsigned int physical_address, unsigned int length, int data) {
    int i;
    int *mb_buffer;
    
    // DDR is accessed through a GP port at offset 0x20000000
    mb_buffer = (int *)(physical_address|0x20000000); // Cast to pointer and convert to DDR offset address

    // Write memory buffer in DDR
    for(i=0; i<length; i++){
        mb_buffer[i]= mb_buffer[i] + data;
    }
}

Initialize the buffer with some values: 

In [5]:
length = 20 
for i in range(length):
    py_buffer[i] = i + 100

Check the content of the buffer

In [8]:
py_buffer[0:length]

PynqBuffer([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
            112, 113, 114, 115, 116, 117, 118, 119])

## Write to buffer from IOP

Write the physical pointer address returned form the allocate instance, along with an initialization value and a length. The IOP application will then write to the memory buffer.

In [ ]:
data = -11
my_function(py_buffer.physical_address, length, data)

Check the contents of the buffer after the IOP application has modified the buffer. The cell above can be re-run with different values of data and length.

In [ ]:
py_buffer[0:length]

## Free the memory

In [ ]:
del py_buffer